In [7]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("nvidia/HelpSteer3", "feedback")

In [ ]:
PROMPT_TEMPLATE = """
    Feedback: {feedback}

    Generate a list of principles that the response is evaluated against in the feedback.
    For each principle, identify a text span from the feedback relating to this principle and then state whether the text span suggests that the response satisfies the principle yes/no/partially.

    Return it as a json dictionary in the format {{"<principle 1>": "<supporting text span>-<yes/no/partially>", "<principle 2>": "<supporting text span>-<yes/no/partially>".}}
"""

In [10]:
train_data = ds['train']

In [11]:
train_data

Dataset({
    features: ['domain', 'language', 'context', 'response1', 'response2', 'feedback1', 'feedback2'],
    num_rows: 38782
})

In [13]:
train_df = pd.DataFrame(train_data)

In [16]:
train_df.keys()

Index(['domain', 'language', 'context', 'response1', 'response2', 'feedback1',
       'feedback2'],
      dtype='object')

In [27]:
idx = 10 

print(f"domain: {train_df['domain'][idx]}")
print("-----------" * 30)
print(f"language: {train_df['language'][idx]}")
print("-----------" * 30)
print(f"context: {train_df['context'][idx]}")
print("-----------" * 30)
print(f"response1: {train_df['response1'][idx]}")
print("-----------" * 30)
print(f"response2: {train_df['response2'][idx]}")
print("-----------" * 30)
print(f"feedback1: {train_df['feedback1'][idx]}")
print("-----------" * 30)


domain: code
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
language: python
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
context: [{'role': 'user', 'content': "i'd like to create a docker-compose file that sets up our net core backend with azurite emulator and a react frontend"}, {'role': 'assistant', 'content': 'To create a `docker-compose.yml` file that sets up your .NET Core backend, Azure Storage Emulator (Azurite), and a